# UNSUPERVISED LEARNING

In [ ]:
def kmeans(X, k=3, max_iter=100, tol=1e-4):
    # Inisialisasi centroid secara acak
    np.random.seed(42)
    random_indices = np.random.choice(X.shape[0], k, replace=False)
    centroids = X[random_indices]

    for i in range(max_iter):
        # Hitung jarak
        distances = np.linalg.norm(X[:, np.newaxis] - centroids, axis=2)
        # Assign cluster
        labels = np.argmin(distances, axis=1)
        
        # Update centroid
        new_centroids = np.array([X[labels == j].mean(axis=0) for j in range(k)])
        
        # Check konvergensi
        if np.linalg.norm(new_centroids - centroids) < tol:
            break
        centroids = new_centroids
    
    return labels, centroids


# SUPERVISED LEARNING

In [ ]:
import numpy as np

class SoftmaxRegression:
    def __init__(self, learning_rate=0.01, n_iter=1000):
        self.learning_rate = learning_rate
        self.n_iter = n_iter
    
    def _softmax(self, Z):
        # Z shape: (n_samples, n_classes)
        # Stabilization trick: Z - np.max(Z, axis=1, keepdims=True)
        expZ = np.exp(Z - np.max(Z, axis=1, keepdims=True))
        return expZ / np.sum(expZ, axis=1, keepdims=True)
    
    def fit(self, X, y):
        # X shape: (n_samples, n_features)
        # y shape: (n_samples,) berisi label kelas (0,1,...,K-1)
        
        n_samples, n_features = X.shape
        self.classes = np.unique(y)
        n_classes = len(self.classes)
        
        # One-hot encoding untuk y
        Y_one_hot = np.zeros((n_samples, n_classes))
        for idx, val in enumerate(y):
            Y_one_hot[idx, np.where(self.classes==val)[0][0]] = 1
        
        # Inisialisasi parameter
        # Tambahkan bias ke X (optional, kita bisa menyertakan bias)
        X = np.hstack([np.ones((n_samples, 1)), X])  # menambahkan kolom ones di depan
        n_features += 1
        
        self.W = np.zeros((n_features, n_classes))  # bobot: (n_features, n_classes)
        
        # Gradient descent
        for i in range(self.n_iter):
            # Forward
            Z = X.dot(self.W)  # (n_samples, n_classes)
            A = self._softmax(Z)  # (n_samples, n_classes)
            
            # Gradient
            grad = (1/n_samples) * X.T.dot(A - Y_one_hot)  # (n_features, n_classes)
            
            # Update weights
            self.W -= self.learning_rate * grad
    
    def predict_proba(self, X):
        # X shape: (n_samples, n_features)
        X = np.hstack([np.ones((X.shape[0], 1)), X])
        Z = X.dot(self.W)
        return self._softmax(Z)
    
    def predict(self, X):
        proba = self.predict_proba(X)
        class_indices = np.argmax(proba, axis=1)
        return self.classes[class_indices]
